# $\mu$-Exercises

## 0) Refresher - Licences

Image that you want to publish some computer program but you want that *only* your classmates of the current "Python P&S" course are allowed to run it. You therefore want to publish your code under a specific licence.
1. On which legal framework could such a licence be based? Write your answer here (double-click to edit): ...
1. Draft a very simple licence which would implement such a scheme. Write your answer here: ...
1. Describe briefely why you think that such a licence would be enforceable by law: what would you tell to the judge if you were called to explain in a court how your licence works? Write your answer here: ...
1. Would such a licence qualify as a *free* licence? Why? Write your answer here: ...

## 1) `async` function
Study the function `count_down`, try to understand what it does and then call the `count_down` function such that you get the expected output!

In [ ]:
import asyncio

async def count_down(name: str, duration: int):
    for i in range(duration):
        print("Countdown {}: {} s".format(name, duration-i))
        await asyncio.sleep(1)
    print("Countdown {} elapsed!".format(name))

# SOLUTION START
...
# SOLUTION END


## 2) Running tasks in parallel
Start two count downs in parallel. They should start counting down at the same time. Make sure one countdown takes 5 seconds to complete and the other takes 10 seconds.

In [ ]:
import asyncio

async def count_down(name: str, duration: int):
    for i in range(duration):
        print("Countdown {}: {} s".format(name, duration-i))
        await asyncio.sleep(1)
    print("Countdown {} elapsed!".format(name))

# SOLUTION START
...
# SOLUTION END

## 3) Run `async/await` from a separate Python script

Run the above count-down timer from a separate Python script!

This needs to be done slightly different than from the Jupyter notebook. (Jupyter is already running an event loop, hence async functions can be called directly from the cells.)

Proceed as follows:

* Create a file in the lecture folder `mu_exercise_3.py`.
* Copy-paste the definition of `count_down` into the script.
* Create an `async def main()` function. Call a single countdown from this main function.
* Use `asyncio.run()` to start the main function.

## 4) Execute tasks in separate threads
It is often useful to execute *blocking* function calls in separate threads such that other concurrent tasks and coroutines are not being blocked.
Run the blocking `user_input` function in the `ThreadPoolExecutor` as it is shown in the lecture.

In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
import time

# Create an 'executor' to run selected tasks on separate threads.
thread_pool = ThreadPoolExecutor()

def user_input():
    # This is a blocking function.
    print("Enter something:")
    return input()

# STUDENT TASK: Execute the function `user_input` in a separate thread.
# Hint: Use the `run_in_executor` function of the event loop.

# START SOLUTION
value = None
# END SOLUTION

print("User input is: ", value)

## 5)  Ping-pong (table tennis) through `asyncio.Queue`s
In the following code two functions are defined: `ping` and `pong`. They are almost the same. Each takes two queues, one for receiving balls and one for sending balls.
Launch this functions in parallel with the right queues in the arguments such that the two tasks can play ping-pong! Because the queues are empty at the beginning it is necessary to manually `put` in a ball in one of the queues.

The output should be:
```
ping
pong
ping
pong
...
```

In [ ]:
import asyncio

async def ping(ball_coming: asyncio.Queue, ball_leaving: asyncio.Queue):
    while True:
        # Wait for a ball to come.
        ball = await ball_coming.get()
        print("ping")
        # Once we have the ball, send it back after some delay.
        await asyncio.sleep(1)
        await ball_leaving.put(ball)

async def pong(ball_coming: asyncio.Queue, ball_leaving: asyncio.Queue):
    while True:
        # Wait for a ball to come.
        ball = await ball_coming.get()
        print("pong")
        # Once we have the ball, send it back after some delay.
        await asyncio.sleep(1)
        await ball_leaving.put(ball)
        
ping_pong_queue = asyncio.Queue() # Used to send balls from ping to pong.
pong_ping_queue = asyncio.Queue() # Used to send balls back from pong to ping.

# STUDENT TASK: Start the two tasks and give them the correct queues.
# Use `asyncio.create_task` to launch both tasks in parallel.
# Then `sleep` for 2 seconds and put in a "Ball" into one of the queues to start the game.

# BEGIN SOLUTION
...
# END SOLUTION


# Exercise 0: Extendable countdown
Bring together the micro exercises and create a countdown that can be prolonged by the user. The countdown should start counting down from 10. While it is counting down the user should be able to enter a durations that will be added to the countdown.

Hints:
* There is already a task for counting down and printing the countdown value. Create a further task which reads the user input and notifies the countdown about the input by using the queue.
* As in the micro-exercise 4 run the `input` function in a separate thread.
* Use a `asyncio.Queue` to send the numbers from the input task to the countdown task.

In [ ]:
import asyncio    
from concurrent.futures import ThreadPoolExecutor

# Create an 'executor' to run selected tasks on separate threads.
thread_pool = ThreadPoolExecutor()


async def count_down(increment_queue: asyncio.Queue):
    """
    Run the countdown.
    If there is a number coming from the `increment_queue` it is used to prolong the countdown.
    """
    counter = 10
    while counter > 0:
        counter = counter - 1
        print("Remaining time: {} s".format(counter))
        await asyncio.sleep(1)
        # Process the queue of increment commands.
        while increment_queue.qsize() > 0:
            i = increment_queue.get_nowait()
            counter = counter + i
    print("Countdown elapsed!")


def user_input():
    """
    Get a string from the user.
    """
    print("Enter something:")
    return input()

# Create the queue that will be used to communicate with the countdown task.
increment_queue = asyncio.Queue()
# Start the countdown task.
countdown_task = asyncio.create_task(count_down(increment_queue))

# STUDENT TASK: Create and start a task that reads the user input and 
# puts it into the `increment_queue` to prolong the countdown.
# START SOLUTION
...
# END SOLUTION

# Wait for the countdown task to terminate.
await countdown_task
# Force the user input task to terminate.
input_task.cancel()


# Exercise 1: Create a chat server
Open the script `ex_01_async_chat_server.py`. It contains almost the same code as the 'echo server' script from the lecture. Right now the script accepts connections, reads incoming data and prints it to the standard output.
Modify the script to build a chat server: Every incoming data should be forwarded to all other open client connections.

Hint: All places where something has to be added are marked with `STUDENT TASK`.

To test the chat server proceed for example as follows:

* Launch the `ex_01_async_chat_server.py` in the terminal.
* Open two other terminals and start `01_async_tcp_client.py` in each of them. This script allows you to write to the chat server and receive messages from the other connections. Alternatively, Linux users are also encouraged to use `netcat` to connect to the chat server.


## Exercise 1.1:  Run the server on the Tardis computers (optional)
Networking is most useful in an actual network. Fortunately, the Tardis computers can reach each other over the network and therefore are suited to test the chat server.
Copy the chat server script and the client script onto a Tardis machine.

Hint: Use sftp and make sure you are logged in into the VPN.

```bash
sftp NETHZ_USERNAME@tardis-cXX.ee.ethz.ch # XX is a number like 01, 02, ...
# SFTP commands:
# Upload: put FILENAME
# Download: get FILENAME
```

Log in on a Tardis machine.

```bash
ssh NETHZ_USERNAME@tardis-cXX.ee.ethz.ch
```

Then launch the server. Don't forget to load **anaconda** first. Otherwise the wrong Python version will be used which leads to errors.

Now log in into other Tardis machines and connect to the machine where the server is running. Also ask some colleagues to test the chat server.

Notice that you will not be able to connect to the chat server running on Tardis machine from the internet due to firewall settings.

# Exercise 2: Chat server with rooms (optional)

Modify the chat server to support multiple independent chat rooms. Change the server script such that the client can join a room by sending the line `/join ROOMNAME`.
Either continue working on your solution from the exercise before or use `ex_02_chat_server.py`.

Hint: Create a dict which holds for every room name all the `StreamWriter` objects that have subscribed to this room. Also the template script contains further hints.

# Exercise 3:  Extend the chat server (optional)

Extend the chat server by other useful functions.


For example:

* Add another command to the chat server to allow users to set a nickname: `/nick`. Then also send along the nickname with the messages.
* Add a `/names` command which sends back the names of the other users in the room.
* *Advanced (requires additional reading)*: Implement end-to-end encrypted chat messages!
    * Check out the `PyNaCl` library: https://pynacl.readthedocs.io/en/stable/ It provides simple yet solid functions for symmetric and asymmetric encryption.
    * Suggestion: Use [symmetric encryption](https://pynacl.readthedocs.io/en/stable/secret/) first based on a password. So in order to exchange encrypted messages all members of a room must know the same password. Use [password hashing](https://pynacl.readthedocs.io/en/stable/password_hashing/) to create a key from the password.
